In [ ]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import multiprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 🚀 Start Timer for Parallel Execution
start_time = time.time()

# 🚀 Enable GPU & Optimizations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True  # Optimize cuDNN operations
torch.backends.cudnn.enabled = True    # Enable cuDNN acceleration

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/HighPerformanceMachineLearning/CrimeDatafrom2020toPresent.csv')

# Convert DATE OCC to datetime format
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], errors='coerce')

# Extract time features
df['Year_OCC'] = df['DATE OCC'].dt.year
df['Month_OCC'] = df['DATE OCC'].dt.month
df['Day_OCC'] = df['DATE OCC'].dt.day
df['Hour_OCC'] = df['TIME OCC'] // 100  # Convert HHMM to hours

# Count crimes per location
crime_counts = df.groupby(['LAT', 'LON']).size()

# Assign Crime Risk Scores (1-10) and scale to 1-5
df['Crime_Risk'] = df.set_index(['LAT', 'LON']).index.map(lambda x: crime_counts.get(x, 0))
df['Crime_Risk'] = np.ceil(10 * (df['Crime_Risk'] / df['Crime_Risk'].max())).astype(int)
df['Crime_Risk'] = df['Crime_Risk'].clip(1, 10)
df['Crime_Risk'] = np.ceil(df['Crime_Risk'] / 2).astype(int)

# Normalize `Crime_Risk` to 0-1
df['Crime_Risk'] = (df['Crime_Risk'] - 1) / 4

# Normalize features
features = ['Year_OCC', 'Month_OCC', 'Day_OCC', 'Hour_OCC', 'LAT', 'LON']
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

# Train-Test Split
train_df, _ = train_test_split(df, test_size=0.2, random_state=42)

# Define Dataset
class CrimeDatasetRegression(Dataset):
    def __init__(self, data):
        self.x = torch.tensor(data[features].values, dtype=torch.float32)
        self.y = torch.tensor(data['Crime_Risk'].values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# 🚀 Auto-adjust DataLoader workers
num_workers = min(4, multiprocessing.cpu_count())  # Set to 4 or half of CPU cores
train_dataset = CrimeDatasetRegression(train_df)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=num_workers, pin_memory=True)

# Define Regression Model
class CrimeRiskRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CrimeRiskRegression, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.output_layer(x)
        return torch.sigmoid(x)

# 🚀 Model Initialization (GPU + Multi-GPU Support)
model = CrimeRiskRegression(input_dim=6, hidden_dim=512).to(device)  # Increased hidden_dim for better GPU usage
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

# 🚀 Loss Function & Optimizer
criterion = nn.HuberLoss(delta=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.005)  # Increased learning rate for faster convergence

# 🚀 Training Loop (Optimized for Speed)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}] - Parallel Loss: {total_loss:.4f}')

# 🚀 End Timer
parallel_time = time.time() - start_time
print(f"🔥 Optimized Parallel Execution Time: {parallel_time:.2f} seconds")



Epoch [1/10] - Parallel Loss: 37.0157
Epoch [2/10] - Parallel Loss: 36.8928
Epoch [3/10] - Parallel Loss: 36.8934
Epoch [4/10] - Parallel Loss: 36.8898
Epoch [5/10] - Parallel Loss: 36.8892
Epoch [6/10] - Parallel Loss: 36.8946
Epoch [7/10] - Parallel Loss: 36.8898
Epoch [8/10] - Parallel Loss: 36.8892
Epoch [9/10] - Parallel Loss: 36.8988
Epoch [10/10] - Parallel Loss: 36.9012
🔥 Optimized Parallel Execution Time: 248.94 seconds


In [ ]:
# Convert predictions from 0-1 scale back to 1-5
def convert_predictions(preds):
    return (preds * 4) + 1  # Scale back to 1-5
  # 🚀 Convert Predictions Back to 1-5 Scale (Parallel Execution)
model.eval()  # Set model to evaluation mode
sample_input = torch.tensor([[0.2, -1.1, 0.5, 0.8, 1.3, -0.5]], dtype=torch.float32).to(device)
output = model(sample_input)
converted_output = convert_predictions(output)

print(f"🔥 Raw Model Output (Parallel): {output.item():.4f}")
print(f"🔥 Converted Prediction (Parallel, 1-5 Scale): {converted_output.item():.2f}")

🔥 Raw Model Output (Parallel): 0.0000
🔥 Converted Prediction (Parallel, 1-5 Scale): 1.00
